<img src='https://user-images.githubusercontent.com/16665629/36117115-329c6220-1041-11e8-98d3-7cd1ce05503c.jpeg'
style="float:right;margin:0 10px 0px 0;width: 150px;">

# Compare observed and theoretical statistical distributions

In [1]:
models = {
            'skymodel.txt': 'pybdsm-meerkat_SourceRecovery-lwimager.lsm.html',
         }
tolerance = 0.0001 # area factor around the source

In [2]:
from ipywidgets import widgets, interact
from IPython.display import Javascript, display
from IPython.display import HTML

def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+5)'))

button = widgets.Button(description="Configure")
button.on_click(run_all)
display(button)

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit"
value="Click here to toggle on/off the raw code."></form>''')

Button(description=u'Configure', style=ButtonStyle())

<IPython.core.display.Javascript object>

In [3]:
import glob
import combat
import matplotlib
import numpy as np
import pylab as plt
from plotly import tools
from scipy.stats import norm
import plotly.graph_objs as go
from plotly import offline as py
from sklearn.metrics import r2_score, mean_squared_error
from plotly.graph_objs import Layout, Figure, XAxis, YAxis
py.init_notebook_mode(connected=True)

/usr/local/lib/python2.7/dist-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [4]:
reload(combat)
ARCSEC_DEG = 1.0/(60*60)
results = combat.property_results(models=models, tolerance=tolerance, input='input')

Loading input/skymodel.txt: ASCII table
Loading input/pybdsm-meerkat_SourceRecovery-lwimager.lsm.html: Tigger sky model
Number of sources recovered: 90


1.3Gb lsmformats(ASCII.py:258:load): Set default freq0 to 1420000000.0 from source on line 2.


In [5]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+12)'))

button = widgets.Button(
    description="PLOT"
)
button.on_click(run_all)
_scaley = widgets.Checkbox(
    value=False,
    description='Same Y-scale',
    disabled=False
)
_scalex = widgets.Checkbox(
    value=False,
    description='Same X-scale',
    disabled=False
)
_num_bins = widgets.IntSlider(
    value=50,
    min=0,
    max=1000,
    step=1,
    description='Number of bins:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [6]:
display(button)
display(_num_bins)

Button(description=u'PLOT', style=ButtonStyle())

IntSlider(value=50, continuous_update=False, description=u'Number of bins:', max=1000)

<IPython.core.display.Javascript object>

## Photometric Measurements

In [7]:
fig = plt.figure()
counter = 0
flux_im = dict()
I_min_max = []
err_I_min_max = []
for input_model, output_model in models.items():
    counter+=1
    name_labels = []
    flux_in_data = []
    flux_out_data = []
    source_scale = []
    phase_center_dist = []
    flux_out_err_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        flux_out_data.append(results[heading]['flux'][n][0])
        flux_out_err_data.append(results[heading]['flux'][n][1])
        flux_in_data.append(results[heading]['flux'][n][2])
        name_labels.append(results[heading]['flux'][n][3])
        phase_center_dist.append(results[heading]['position'][n][-3])
        source_scale.append(results[heading]['shape'][n][3])
    LIMIT = 0
    zipped_props = zip(flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale)
    flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    I_min_max += flux_out_data
    err_I_min_max += flux_out_err_data
    flux_R_score = r2_score(flux_in_data, flux_out_data)
    flux_MSE = mean_squared_error(flux_in_data, flux_out_data)
    I_out_in = [float(I_out)/I_in for I_out,I_in in zip(flux_out_data,flux_in_data)]
    n, bins, patches = matplotlib.pyplot.hist(I_out_in, _num_bins.value, normed=False, facecolor='blue',
                                           orientation='horizontal', alpha=0.5)
    (mu, sigma) = norm.fit(I_out_in)
    peak = [bins[i] for i,v in enumerate(n) if v == n.max()][0]
    print  "Model: %s \n%s \nMean: %.4f | Mean Squared Error: %.2E | R2: %.4f Peak: %.4f | STD: %.4f \n%s" % (
        heading, '-'*45, mu, flux_MSE, flux_R_score, peak, sigma, '='*85)
    counter += 1
plt.close(fig)

Model: pybdsm-meerkat_SourceRecovery-lwimager 
--------------------------------------------- 
Mean: 0.9762 | Mean Squared Error: 5.44E-09 | R2: 0.9967 Peak: 0.9749 | STD: 0.0155 


In [8]:
data = []
im_titles =[]
flux_im = dict()
BINS = _num_bins.value
for input_model, output_model in models.items():
    header = output_model[:-9]
    im_titles.append('<b>%s flux density</b>'%header.upper())
    im_titles.append('<b>%s flux density</b>                                        '%header.upper())

fig = tools.make_subplots(rows=len(models.keys()), cols=2, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.005, vertical_spacing = 0.15,
                          subplot_titles=im_titles)
i = -1
counter = 0
flux_im = dict()
I_min_max = []
err_I_min_max = []
for input_model, output_model in models.items():
    i += 1
    counter += 1
    name_labels = []
    flux_in_data = []
    flux_out_data = []
    source_scale = []
    phase_center_dist = []
    flux_out_err_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        flux_out_data.append(results[heading]['flux'][n][0])
        flux_out_err_data.append(results[heading]['flux'][n][1])
        flux_in_data.append(results[heading]['flux'][n][2])
        name_labels.append(results[heading]['flux'][n][3])
        phase_center_dist.append(results[heading]['position'][n][-3])
        source_scale.append(results[heading]['shape'][n][3])
    LIMIT = 0
    zipped_props = zip(flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale)
    flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    flux_R_score = r2_score(flux_in_data, flux_out_data)
    flux_MSE = mean_squared_error(flux_in_data, flux_out_data)
    I_out_in = [float(I_out)/I_in for I_out,I_in in zip(flux_out_data,flux_in_data)]
    axis_min_max = combat.axis_min_max(I_min_max, error=err_I_min_max, tolerance=0.01)
    fig.append_trace(go.Scatter(x=np.array(flux_in_data), y=np.array(flux_out_data),
                                mode = 'markers', showlegend=False,
                                text = name_labels, name = '%s flux_ratio' % heading,
                                marker = dict(color = phase_center_dist, showscale=True, colorscale='Jet',
                                              reversescale=False,
                                              colorbar = dict(title='Distance from phase center (arcsec)',
                                                              titleside ='right', titlefont=dict(size=16), x = 0.82)
                                             ) if i == 0 else dict(color = phase_center_dist, colorscale='Jet',
                                                                   reversescale=False),
                                error_y=dict(type='data', array=flux_out_err_data,
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Histogram(x=I_out_in, histnorm='count', nbinsx=BINS, name='%s' % heading,
                                  marker = dict(color = 'rgb(64, 98, 221)'), showlegend=False), i+1, 2)
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', plot_bgcolor='rgb(229,229,229)',
                         legend=dict(x=0.8,y=1.0),)
    fig['layout'].update(
        {'yaxis{}'.format(counter):YAxis(title=u'I_out',gridcolor='rgb(255,255,255)',
                                        # titlefont=dict(size=18),
                                         range=axis_min_max if _scaley.value else [],
        #range=[1,10],
        tickfont=dict(size=15),
        titlefont=dict(size=17),
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update({'xaxis{}'.format(counter+i):XAxis(title='I_in', position=0.0,
                                                            titlefont=dict(size=17),
                                                            overlaying='x')})
    fig['layout'].update({'xaxis{}'.format(counter+i+1):XAxis(title='Counts',
                                                              titlefont=dict(size=17),
                                                              domain=[0.505, 0.8])}
        )
py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Astrometric Measurements

In [9]:
counter = 0
pos_min_max = []
DEC_off_min_max = []
RA_off_min_max = []
for input_model, output_model in models.items():
    counter+=1
    RA_offset = []
    DEC_offset = []
    DELTA_PHASE0 = []
    source_labels = []
    flux_in_data = []
    delta_pos_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        delta_pos_data.append(results[heading]['position'][n][0])
        RA_offset.append(results[heading]['position'][n][1])
        DEC_offset.append(results[heading]['position'][n][2])
        DELTA_PHASE0.append(results[heading]['position'][n][3])
        flux_in_data.append(results[heading]['position'][n][4])
        source_labels.append(results[heading]['position'][n][5])
    zipped_props = zip(delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels)
    delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    pi,sin,cos = np.pi,np.sin,np.cos
    r1, r2 = np.array(RA_offset).std(), np.array(DEC_offset).std()
    theta = np.linspace(0,2*pi,45)
    x1 = r1*cos(theta)
    y1 = r2*sin(theta)
    print "%s\n1 sigma = %s\n%s"%(heading, np.sqrt(r1*r2), 25*'=')

pybdsm-meerkat_SourceRecovery-lwimager
1 sigma = 0.0005913046214056889


In [10]:
data = []
im_titles =[]
for input_model, output_model in models.items():
    header = output_model[:-9]
    im_titles.append('<b>%s Position</b>'%header.upper())
    im_titles.append('<b>%s Position</b>'%header.upper())

fig = tools.make_subplots(rows=len(models.keys()), cols=2, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.25,
                          vertical_spacing = 0.15,
                          subplot_titles=im_titles)
counter = 0
for input_model, output_model in models.items():
    counter+=1
    RA_offset = []
    DEC_offset = []
    DELTA_PHASE0 = []
    source_labels = []
    flux_in_data = []
    delta_pos_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        delta_pos_data.append(results[heading]['position'][n][0])
        RA_offset.append(results[heading]['position'][n][1])
        DEC_offset.append(results[heading]['position'][n][2])
        DELTA_PHASE0.append(results[heading]['position'][n][3])
        flux_in_data.append(results[heading]['position'][n][4])
        source_labels.append(results[heading]['position'][n][5])
    zipped_props = zip(delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels)
    delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    fig.append_trace(go.Scatter(x=np.array(flux_in_data), y=np.array(delta_pos_data),
                                mode = 'markers', showlegend=False,
                                text = source_labels, name = '%s flux_ratio' % header,
                                marker = dict(color = DELTA_PHASE0, showscale=True,
                                              colorscale='Jet', reversescale=True,
                                              colorbar = dict(title= 'Distance from phase centre (arcsec)',
                                                              titleside ='right', titlefont=dict(size=16), x = 0.4)
                                             ) if i == 0 else dict(color = DELTA_PHASE0, colorscale='Jet',
                                                                   reversescale=True),
                                error_y=dict(type='data', #array=flux_out_err_data,
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=np.array(RA_offset), y=np.array(DEC_offset),
                                mode = 'markers', showlegend=False,
                                text = source_labels, name = '%s flux_ratio' % heading,
                                marker = dict(color = flux_out_data, showscale=True, colorscale='Viridis', reversescale=True,
                                              colorbar = dict(title= 'Input Flux', titleside ='right',
                                                             titlefont=dict(size=16))
                                             ) if i == 0 else dict(color = flux_out_data, colorscale='Viridis',
                                                                   reversescale=True),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 2)
    r1, r2 = np.array(RA_offset).std(), np.array(DEC_offset).std()
    theta = np.linspace(0,2*pi,45)
    x1 = r1*cos(theta)
    y1 = r2*sin(theta)
    #print "%s\n1 sigma = %s\n%s"%(imager, np.sqrt(r1*r2), 25*'=')
    fig.append_trace(go.Scatter(x=x1, y=y1,
                                mode = 'lines', showlegend=True if i == 0 else False,
                                name = '1σ',
                                text = '1σ ~ %f' % (np.sqrt(r1*r2)),
                                marker = dict(color = 'rgb(0, 0, 255)')), i+1, 2)
    fig.append_trace(go.Scatter(x=2*x1, y=2*y1,
                                mode = 'lines', showlegend=True if i == 0 else False,
                                name = '2σ',
                                text = '2σ ~ %f' % (2*np.sqrt(r1*r2)),
                                marker = dict(color = 'rgb(255, 0, 0)')), i+1, 2)
    fig.append_trace(go.Scatter(x=3*x1, y=3*y1,
                                mode = 'lines', showlegend=True if i == 0 else False,
                                name = '3σ',
                                text = '3σ ~ %f' % (3*np.sqrt(r1*r2)),
                                marker = dict(color = 'rgb(255, 255, 0)')), i+1, 2)
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', plot_bgcolor='rgb(229,229,229)',
                         legend=dict(xanchor=True, x=1.1,y=1)
                        )
    fig['layout'].update(
        {'yaxis{}'.format(counter+i):YAxis(title=u'Delta position [arcsec]',gridcolor='rgb(255,255,255)',
                                           color='rgb(0,0,0)',
                                           range=axis_min_max if _scaley.value else [],
        #range=[1,10],
        tickfont=dict(size=14, color='rgb(0,0,0)'),
        titlefont=dict(size=15),
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update(
        {'yaxis{}'.format(counter+i+1):YAxis(title='Declination offset [arcsec]',gridcolor='rgb(255,255,255)',
                                             color='rgb(0,0,0)',
                                             range=dec_min_max if _scaley.value else [],
        #range=[1,10],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=17),
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update({'xaxis{}'.format(counter+i):XAxis(title='Input Flux',
                                                            titlefont=dict(size=17),
                                                            zeroline=False, position=0.0, overlaying='x',)})
    fig['layout'].update({'xaxis{}'.format(counter+i+1):XAxis(title='Right Ascension offset [arcsec]',
                                                              titlefont=dict(size=17),
                                                              range=ra_min_max if _scalex.value else [],
                                                              zeroline=False)}# domain=[0.505, 0.8])}
        )

py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Morphological Measurements

In [11]:
data = []
im_titles =[]
for input_model, output_model in models.items():
    header = output_model[:-9]
    im_titles.append('<b>%s Source shape</b>'%header.upper())
    im_titles.append('<b>%s Source shape</b>'%header.upper())

fig = tools.make_subplots(rows=len(models.keys()), cols=2, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.1,
                          vertical_spacing = 0.15,
                          subplot_titles=im_titles)
counter = 0
for input_model, output_model in models.items():
    counter+=1
    SCALE = []
    SCALE_ERR = []
    flux_in_data = []
    DELTA_PHASE0 = []
    source_labels = []
    MAJ_MIN_angle_in = []
    MAJ_MIN_angle_out = []
    heading = output_model[:-9]
    for n in range(len(results[header]['flux'])):
        MAJ_MIN_angle_out.append(results[header]['shape'][n][0])
        MAJ_MIN_angle_in.append(results[header]['shape'][n][2])
        SCALE.append(results[header]['shape'][n][3])
        SCALE_ERR.append(results[header]['shape'][n][4])
        flux_in_data.append(results[header]['shape'][n][5])
        source_labels.append(results[header]['shape'][n][6])
        DELTA_PHASE0.append(results[header]['position'][n][3])  
    zipped_props = zip(MAJ_MIN_angle_out, MAJ_MIN_angle_in, DELTA_PHASE0, SCALE, SCALE_ERR)
    MAJ_MIN_angle_out, MAJ_MIN_angle_in, DELTA_PHASE0, SCALE, SCALE_ERR = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    maj_in = [maj_min_angle_in[0] for maj_min_angle_in in MAJ_MIN_angle_in]
    maj_out = [maj_min_angle_out[0] for maj_min_angle_out in MAJ_MIN_angle_out]
    min_in = [maj_min_angle_in[1] for maj_min_angle_in in MAJ_MIN_angle_in]
    min_out = [maj_min_angle_out[1] for maj_min_angle_out in MAJ_MIN_angle_out]
    angle_offset = [(maj_min_angle_out[2] - maj_min_angle_in[2]) for maj_min_angle_out, maj_min_angle_in in zip(
        MAJ_MIN_angle_out, MAJ_MIN_angle_in)]
    fig.append_trace(go.Scatter(x=maj_in, y=maj_out, mode = 'markers', showlegend=False,
                                text = name_labels, name = '%s flux_ratio' % header,
                                marker = dict(color = flux_in_data, showscale=True, colorscale='Viridis', reversescale=True,
                                              colorbar = dict(title= 'Signal-to-Noise ratio', titleside ='right',
                                                             titlefont=dict(size=16))
                                             ) if i == 0 else dict(color = flux_in_data, colorscale='Viridis',
                                                                   reversescale=True),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=min_in, y=min_out,
                                mode = 'markers', showlegend=False,
                                text = name_labels, name = '%s flux_ratio' % header,
                                marker = dict(color = flux_in_data, showscale=True,
                                              colorscale='Viridis', reversescale=True,
                                              colorbar = dict(title= 'Distance from phase center', titleside ='right',
                                                             titlefont=dict(size=16))
                                             ) if i == 0 else dict(color = DELTA_PHASE0, colorscale='Viridis',
                                                                   reversescale=True),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 2)
    pi,sin,cos = np.pi,np.sin,np.cos
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', plot_bgcolor='rgb(229,229,229)',
                         legend=dict(xanchor=True)
                        )
    fig['layout'].update(
        {'yaxis{}'.format(counter+i):YAxis(title=u'Output major axis[arcsec]',gridcolor='rgb(255,255,255)',
                                           color='rgb(0,0,0)',
        #range=[1,10],
        tickfont=dict(size=14, color='rgb(0,0,0)'),
        titlefont=dict(size=17),
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update(
        {'yaxis{}'.format(counter+i+1):YAxis(title='Ouput minor axis[arcsec]',gridcolor='rgb(255,255,255)',
                                             color='rgb(0,0,0)',
        #range=[1,10],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=17),
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update({'xaxis{}'.format(counter+i):XAxis(title='Input major axis[arcsec]',
                                                            titlefont=dict(size=17),
                                                            zeroline=False, position=0.0, overlaying='x',)})
    fig['layout'].update({'xaxis{}'.format(counter+i+1):XAxis(title='Input minor axis[arcsec]',
                                                              titlefont=dict(size=17),
                                                              zeroline=False)}# domain=[0.505, 0.8])}
        )

py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Source meta-data

In [12]:
src_nm = widgets.Text(
    value='',
    placeholder='ay040',
    description='Source Name:',
    disabled=False
)
display(src_nm)

Text(value=u'', description=u'Source Name:', placeholder=u'ay040')

In [13]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+2)'))

button = widgets.Button(description="Search")
button.on_click(run_all)
display(button)

Button(description=u'Search', style=ButtonStyle())

<IPython.core.display.Javascript object>

In [14]:
src_target = src_nm.value
for k,v in sorted(results.items()):
    for i, src in enumerate(results[k]['flux']):
        #import IPython; IPython.embed()
        if src_target == src[-1]:
            #pos_in = results[k]['position'][i][1]
            #pos_out = results[k]['position'][i][2]
            #print k
            print ('============')
            print ("Source info:\nname = %s"
                   "\nIn_flux = %s Jy\nOut_flux = %s Jy"
                   "\nOut_flux_err = %s Jy"
                   "\nModel_file = %s"
                   % (src[3], round(src[2], 6), round(src[0], 6), round(src[1], 6), k))
            print ('============')
            #print ("Source info:\nname = %s\nIn_RA = %s deg\nIn_DEC = %s deg"
            #       "\nOut_RA = %s deg\nOut_DEC = %s deg"
            #       "\nIn_flux = %s Jy\nOut_flux = %s Jy\nIn_Out_flux_ratio = %s"
            #       "\nSNR = %s"
            #       "\nOut_flux_err = %s Jy\nI_peak = %s Jy\nI_peak_err = %s Jy"
            #       "\nRestored_image = %s \nMSdir = %s"
            #       % (src[-4], pos_in[0], pos_in[1], pos_out[0], pos_out[1],
            #       round(src[2], 6), round(src[0], 6), round(src[-3], 6), round(src[4], 6),
            #       round(src[1], 6), round(src[-2], 6), round(src[-1], 6),
            #       results[k]['restored_image'], results[k]['msfile']))
            print '\n'

Source info:
name = ci082
In_flux = 0.000857 Jy
Out_flux = 0.000835 Jy
Out_flux_err = 4.7e-05 Jy
Model_file = pybdsm-meerkat_SourceRecovery-lwimager


